## Part II: Broker API and Input data
To move from backtest to execution, signals must be converted into broker-formated orders. This involves identifying entries and exits, sizing positions, and ensuring all orders respect risk limits. This section describes the workflow that connects the backtesting engine to a broker for live trading.

We use a daily cycle: signals are generated after the market close, checked against current data, and exported in a broker-compatible format for execution on the following session.

### 10. Broker Integration and Trade Deployment Workflow
A daily process links Zipline Reloaded with Interactive Brokers (IB) via the TWS Basket Trader. The script runs after the close, using updated end-of-day data from Norgate. It identifies new trades, applies sizing and validation rules, and exports the results as a structured CSV file.

For portfolios with 70+ futures contracts, this avoids manual entry while still allowing pre-trade review. Each row in the file specifies trade direction, quantity, symbol, expiry, exchange, limit price, and other required fields.

This fits the end-of-day model: new data is processed, signals are evaluated, and the system outputs a complete trade list. The workflow leaves room for discretionary checks while keeping execution systematic.

The workflow runs each evening at the CME Globex daily session close — 16:00 Chicago (22:00 UTC in winter, 21:00 UTC in summer) — which is used as the reference since the majority of traded futures are listed there.

1. **Data Update and Inception Re-Run**

Each day, the full futures dataset is ingested from Norgate into Zipline Reloaded. While this isn’t delta-efficient, it's necessary due to how Zipline manages internal state. After ingesting the data, the entire strategy is re-executed from inception to the most recent close. 

2. **Trade Extraction and Conversion to IBKR Basket Format**

Upon completion, the system parses `transactions.csv` and filters for trades dated to the last bar of the backtest. These trades are treated as actionable signals for the next trading session after particular adjustments.

3. **Validation with Live Market Data and Execution Audit**

A report is generated summarizing the trades for next day. Position sizing, rollovers, and price discrepancies are highlighted. Each trade is validated using current IB market data. Volatility and stop distances are recalculated. FX rates are applied where needed. If the limit price deviates by more than ±0.5% from the reference price, the trade is flagged for review. This serves as a final check before submitting the trades to IB.

In the following sections, we outline the code and logic used to generate the order basket for execution via TWS.

### 10.1. Data Update and Inception Re-Run
To ensure the backtest includes the most recent market data, the first step in the daily routine is to rebuild the Zipline data bundle. Since Zipline Reloaded doesn’t support partial or incremental ingestion for Norgate futures, the entire dataset must be reprocessed from scratch.

This starts by deleting the existing bundle directory (_.zipline/data/norgatedata-full-futures_) using `shutil.rmtree()`. This clears all compiled files and cached data associated with the bundle. After that, running the ingestion step is preferable from an Anaconda Prompt rather than inside the script, since zipline ingest is a command-line tool and can behave more reliably when executed directly in the environment shell: `zipline ingest -b norgatedata-full-futures`.

In [ ]:
import shutil
import os
import subprocess

# Step 1: Delete the old bundle folder
bundle_path = r"C:\Users\Juanan\.zipline\data\norgatedata-full-futures"

if os.path.exists(bundle_path):
    shutil.rmtree(bundle_path)
    print(f"🧹 Deleted bundle directory: {bundle_path}")
else:
    print("ℹ️ Bundle directory not found. Proceeding to ingest...")

Although inefficient (only one or two new bars are typically added), full re-ingestion is necessary due to Zipline’s internal handling of futures data, roll adjustment, and continuous contract reconstruction. A delta ingestion method would be faster but isn’t supported at this time.

After ingestion, the full backtest is re-run using the updated data. The capital base is fixed at $1,000,000, which matches the Interactive Brokers demo account used for deployment. This keeps position sizing aligned with the live environment and avoids introducing capital compounding into signal generation. The backtest window starts at the model’s inception (`2025-08-15`) and ends with the most recent session (e.g., `2025-08-18`).

Running the full model each day ensures:

- Consistent portfolio tracking

- Proper trailing stop evaluation

- No indicator drift

- Signals are based entirely on historical context

After the model finishes, trades from the most recent bar are pulled from Zipline `transactions-YYYYMM(start)-YYYYMM(end)-X(model).csv` and passed to the order generation process.

In [ ]:
import os

paths = [
    r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\2.Deployment_Backtest",
    r"C:\Users\Juanan\Downloads\CQF\2.Mean_Reversion\1.Zipline_Trades\2.Deployment_Backtest"
]

for path in paths:
    print(f"Contents of: {path}")
    if os.path.exists(path):
        for item in os.listdir(path):
            print("  ", item)
    else:
        print("  Path does not exist.")
    print()

For efficiency, the workflow is presented here using the trend-following model. The mean-reversion model runs through the same daily pipeline—data ingestion, re-run, extraction, validation, and basket construction—differing only in trading logic. A quick look at the `transactions_2025-08-15_to_2025-08-18_TF.csv` file produced by the trend-following model:

In [ ]:
import pandas as pd

# === File path ===
tf_returns_path = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\2.Deployment_Backtest\transactions_2025-08-15_to_2025-08-18_TF.csv"

# === Read CSV files ===
df_tf = pd.read_csv(tf_returns_path)

# === Display helper ===
def preview_df(df, title, rows=5):
    print(f"\n{'='*60}")
    print(f"{title:^60}")
    print(f"{'='*60}")
    display(df.head(rows))  # Nicer than print in Jupyter / IPython
    print("\n")

# === Show previews ===
preview_df(df_tf, "📈 Futures Trend Following (TF) Returns")

### 10.2. Trade Extraction and Conversion to IBKR Basket Format
After the backtest completes, this step extracts and formats the model’s executed trades from the most recent trading day into a structure compatible with Interactive Brokers’ TWS Basket Trader. The process begins by loading the `transactions-YYYYMM(start)-YYYYMM(end)-X(model).csv` file generated by Zipline, which contains the full trade history. The script identifies the latest trading day in the file and filters only the trades from that specific session. This ensures that only new positions (entries or exits) triggered by the model are considered for execution.

To match Zipline’s internal futures identifiers with the contract specs required by Interactive Brokers, a local mapping file named `ZiplineIB_Mapping.xlsx` is used. This file defines each root symbol (e.g., ES, ZB) alongside its IBKR contract symbol, exchange, currency, tick size, and contract multiplier.

Below we can see the components needed to make this first adjustment for IB.

In [ ]:
import pandas as pd

file_path = r"C:\Users\Juanan\Downloads\CQF\ZiplineIB_Mapping.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Display the first few rows
df.head()

Each trade record is then parsed to extract its full contract symbol, root symbol, and expiry in _YYYYMM_ format. This is done using string manipulation based on the standard futures code structure (e.g., _NKDU25_ becomes root _NKD_ and expiry _202509_). These details are merged with a custom symbol mapping file that links Zipline root symbols to their corresponding Interactive Brokers tickers, exchanges, currencies, and tick sizes.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os

# === CONFIGURATION ===
transactions_path = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\2.Deployment_Backtest\transactions_2025-08-15_to_2025-08-18_TF.csv"
mapping_path = r"C:\Users\Juanan\Downloads\CQF\ZiplineIB_Mapping.xlsx"
output_dir = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\2.Zipline_IB_Trades"
ib_account = "DU9355692"

# === LOAD FILES ===
transactions_df = pd.read_csv(transactions_path)
mapping_df = pd.read_excel(mapping_path, sheet_name="ZiplineIB")

# === LOAD AND USE ALL TRANSACTIONS FOR TESTING ===
transactions_df['dt'] = pd.to_datetime(transactions_df['dt'])

# Detect last trading day in the file
transactions_df['dt'] = pd.to_datetime(transactions_df['dt'])
last_trading_day = transactions_df['dt'].dt.date.max()
signal_date_str = last_trading_day.strftime("%Y-%m-%d")

# Filter only trades from last trading day
transactions_df = transactions_df[transactions_df['dt'].dt.date == last_trading_day]

print(f"📆 Using transactions from last trading day: {signal_date_str}")


# === FUTURES MONTH CODES ===
month_map = {
    'F': '01', 'G': '02', 'H': '03', 'J': '04', 'K': '05', 'M': '06',
    'N': '07', 'Q': '08', 'U': '09', 'V': '10', 'X': '11', 'Z': '12'
}

# === EXTRACT SYMBOL INFO ===
def extract_symbol_info(symbol_str):
    try:
        code = symbol_str.split('[')[-1].split(']')[0]  
        month_code = code[-3]
        year = '20' + code[-2:]
        month = month_map.get(month_code.upper(), '00')
        expiry = f"{year}{month}"
        root_symbol = ''.join(filter(str.isalpha, code[:-3]))
        return pd.Series([code, root_symbol, expiry])
    except:
        return pd.Series([None, None, None])

transactions_df[['FullSymbolCode', 'RootSymbol', 'Expiry']] = transactions_df['symbol'].apply(extract_symbol_info)

# === MERGE WITH MAPPING FILE ===
merged_df = transactions_df.merge(mapping_df, left_on='RootSymbol', right_on='Symbol', how='left')

# === ROUND TO TICK SIZE ===
def extract_tick_size(val):
    try:
        return float(str(val).split()[0])
    except:
        return 0.01

merged_df['TickSize'] = merged_df['Tick Size'].apply(extract_tick_size)
merged_df['RoundedPrice'] = (merged_df['price'] / merged_df['TickSize']).round() * merged_df['TickSize']


# === NO SPECIAL PRICE ADJUSTMENT NEEDED ===
merged_df['AdjustedPrice'] = merged_df['RoundedPrice']


# === BUILD FINAL TWS BASKET DATAFRAME ===
tws_df = pd.DataFrame({
    'dt': merged_df['dt'],
    'Action': merged_df['amount'].apply(lambda x: 'BUY' if x > 0 else 'SELL'),
    'Quantity': merged_df['amount'].abs(),
    'Symbol': merged_df['Symbol IB'],
    'SecType': merged_df['SecType'],
    'LastTradingDayOrContractMonth': merged_df['Expiry'],
    'DivPrt': False,
    'Exchange': merged_df['Exchange IB'],
    'Quotation': merged_df['Quotation'],
    'Currency': merged_df['Currency IB'],
    'TimeInForce': 'DAY',
    'OrderType': 'LMT',
    'LmtPrice': merged_df['AdjustedPrice'],
    'BasketTag': signal_date_str,
    'Account': ib_account,
    'OrderRef': f"{signal_date_str}_SignalZipline",
    'Multiplier': merged_df['Contract Size IB']
})

# === SAVE TO FILE ===
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"TWS_Basket_{signal_date_str}.csv")
tws_df.to_csv(output_file, index=False)


print(f"\n✅ TWS Basket for {signal_date_str} saved to:\n{output_file}")

As we can see in the code above, once the trades are matched to their execution specs, prices are rounded to the correct tick size, and formatted for limit orders. No additional slippage or spread adjustments are applied at this stage. A new DataFrame is built to represent the final TWS basket, including order type (`LMT`), quantity, symbol, expiry, and account ID. The trades are then saved to a CSV file named `TWS_Basket_<date>.csv`, which later will be automatically executed through IBKR’s TWS API after proper adjustments and validations.

In [ ]:
from pathlib import Path
import pandas as pd

folder = Path(r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\2.Zipline_IB_Trades")

# Find all CSVs in the folder
csv_files = list(folder.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in: {folder}")

# Pick the most recently *created* CSV (on Windows, st_ctime is creation time)
latest_csv = max(csv_files, key=lambda p: p.stat().st_ctime)

print(f"Loading: {latest_csv.name}")
# UTF-8 with BOM is common from Excel exports; this handles it gracefully
df = pd.read_csv(latest_csv, encoding="utf-8-sig")

from IPython.display import display

display(df.head(20))

### 10.3. Validation with Live Market Data and Execution Audit

Before any orders are submitted, all trades flagged by the strategy—including exits, size mismatches, or risk-level adjustments—are reviewed in a final report. This serves both as a verification step and a way to preserve an audit trail. No trades are pushed to TWS until this check is complete. The process is split by action type, starting with the exits.

#### 10.3.1. Execution Audit – Sell Orders

This section handles validation for SELL-side instructions. These are typically exits triggered by stop levels, trend reversals, or roll conditions. Since these signals are based on end-of-day bars but trades are submitted the next day, price drift needs to be checked before sending orders.

The process begins by loading the latest trade basket CSV—exported earlier by the backtest pipeline—and filtering for rows marked as _SELL_. The goal is to confirm whether the model’s exit logic still holds under current market prices.

Each entry is converted into an IBKR futures contract using key fields: root symbol, expiry (`YYYYMM`), exchange, currency, and multiplier. Using the IB API, the script connects to TWS and requests and compares the most recent closing price from IB with the `LmtPrice` from the basket file. 

The comparison produces a percentage deviation (`PctDiff`). If the difference between the IB price and the model’s price exceeds ±0.5%, the order is flagged (`Alert_Deviation`). This is a basic safeguard against executing trades where price context has shifted—e.g., overnight rallies, extended moves, or invalid trailing stops.

All trades are compiled into a table with calculated fields: contract metadata, current close, deviation vs. model, and any alert triggers. The final output is displayed as a scrollable table in Jupyter.

In [ ]:
import os
import glob
import time
import pandas as pd
import numpy as np
import threading
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData
from IPython.display import display, HTML

# === IB API Wrapper ===
class MyTradingStrategy(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = {}
        self.received = set()

    def historicalData(self, reqId: int, bar: BarData):
        if 'closes' not in self.data[reqId]:
            self.data[reqId]['closes'] = []
        self.data[reqId]['closes'].append(bar.close)

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        self.received.add(reqId)

# === Contract Builder ===
def create_futures_contract(symbol, last_trade_date, exchange, currency, multiplier):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "FUT"
    contract.lastTradeDateOrContractMonth = str(last_trade_date)[:6]
    contract.exchange = exchange
    contract.currency = currency
    contract.multiplier = str(int(float(multiplier)))
    return contract

# === Load Latest TWS Basket ===
tws_path = sorted(glob.glob(r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\2.Zipline_IB_Trades\TWS_Basket_*.csv"))[-1]
tws_df = pd.read_csv(tws_path)
tws_df['LastTradingDayOrContractMonth'] = tws_df['LastTradingDayOrContractMonth'].astype(str).str[:6]
tws_df['Multiplier'] = tws_df['Multiplier'].astype(str)

# === Filter SELL Orders Only ===
sells_df = tws_df[tws_df['Action'].str.upper() == 'SELL'].reset_index(drop=True)

# === Start IB Connection ===
app = MyTradingStrategy()
app.connect("127.0.0.1", 7497, clientId=65)
threading.Thread(target=app.run, daemon=True).start()
time.sleep(1)

# === Process SELL Orders ===
def process_sell_side(df_side):
    if df_side.empty:
        display(HTML("<b>No SELL orders found.</b>"))
        return pd.DataFrame()

    app.data.clear()
    app.received.clear()

    for reqId, row in df_side.iterrows():
        contract = create_futures_contract(
            row["Symbol"],
            row["LastTradingDayOrContractMonth"],
            row["Exchange"],
            row["Currency"],
            row["Multiplier"]
        )
        app.data[reqId] = {
            "symbol": row["Symbol"],
            "exchange": row["Exchange"],
            "contract_month": row["LastTradingDayOrContractMonth"],
            "currency": row["Currency"],
            "multiplier": float(row["Multiplier"]),
            "closes": []
        }
        app.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime="",
            durationStr="21 D",
            barSizeSetting="1 day",
            whatToShow="TRADES",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[]
        )

    # Wait for all data
    timeout = 60
    start_time = time.time()
    while len(app.received) < len(df_side) and time.time() - start_time < timeout:
        time.sleep(0.5)

    # Process results
    rows = []

    for reqId, info in app.data.items():
        closes = info.get("closes", [])
        if len(closes) >= 1:
            close = closes[-1]

            row = df_side.iloc[reqId].to_dict()
            quotation = row.get("Quotation", "")
            symbol = row.get("Symbol", "")
            exchange = row.get("Exchange", "")

            # Adjust close and LmtPrice for cents-based symbols
            if symbol in ["ZR", "HG", "SI"]:
                row["LmtPrice"] /= 100
                close /= 100
            elif isinstance(quotation, str) and quotation.lower().startswith("cents per"):
                close /= 100

            lmt = row["LmtPrice"]
            pct_diff = ((close - lmt) / lmt) if lmt not in (0, None) else np.nan

            result = {
                "Action": row["Action"],
                "Quantity": row["Quantity"],
                "Symbol": symbol,
                "Exchange": exchange,
                "LastTradingDayOrContractMonth": row["LastTradingDayOrContractMonth"],
                "Currency": row["Currency"],
                "OrderType": row["OrderType"],
                "Multiplier": float(row["Multiplier"]),
                "Quotation": quotation,
                "LmtPrice": lmt,
                "Close": close,
                "PctDiff": round(pct_diff * 100, 2) if pd.notnull(pct_diff) else None,
                "Alert_Deviation": abs(pct_diff) > 0.5 if pd.notnull(pct_diff) else False,
            }
            rows.append(result)

    return pd.DataFrame(rows)

# === Process and Display ===
part4_df = process_sell_side(sells_df)

# === Define Highlighting ===
group1 = ['Action', 'Quantity', 'Symbol', 'Exchange', 'LastTradingDayOrContractMonth', 'Currency', 'OrderType', 'Multiplier', 'Quotation']
group2 = ['LmtPrice', 'Close', 'PctDiff']
group3 = ['Alert_Deviation', 'Alert_MissingPrice']

def highlight_columns(val, colname):
    if colname in group1:
        return 'background-color: #e6f7ff'
    elif colname in group2:
        return 'background-color: #fff7e6'
    elif colname in group3:
        return 'background-color: #ffe6cc'
    else:
        return ''

# === Show Final Table ===
if not part4_df.empty:
    styled_sell = part4_df.style.apply(lambda row: [highlight_columns(row[c], c) for c in part4_df.columns], axis=1)
    scrollable_sell = f"""
    <div style="overflow-x: auto; white-space: nowrap; font-family: monospace; font-size: 13px;">
        {styled_sell.to_html()}
    </div>
    """
    display(HTML("<h3>📊 PART 4 (SELLs only): Execution Review</h3>"))
    display(HTML(scrollable_sell))

In [ ]:
app.disconnect()

There are no closing orders in these files. The deployment period is short, so no stop-loss exits or contract rollovers have been triggered yet. Still, the code shows that futures positions are being identified correctly. 

The following routine is responsible for closing live open futures positions held in the Interactive Brokers account. It’s used when we want to enforce a full exit from the market—whether for risk-off scenarios, rollovers or end-of-session flat requirements. The function connects to IB, queries live open positions, cross-references missing exchange fields using the latest Zipline-derived TWS basket, and issues market orders to flatten them.

The script begins by loading the most recent TWS basket CSV file from the local export directory. This file contains all model-generated trades, including the most accurate contract metadata available at signal generation time. From this, it builds a lookup table keyed by (`Symbol`, `Expiry`, `Currency`, `Multiplier`) to resolve missing exchange identifiers later in the process.

Next, a custom API client (`IBPositionCloser`) connects to TWS via the IB Python API. Once connected, it requests all current open positions. The wrapper filters for futures contracts only and excludes only positions matching by (`Symbol`, `Expiry`, `Currency`, `Multiplier`) in our `TWS_Basket_*.csv`.

After validation, each open position is converted into a market order in the opposite direction. Long positions are closed via SELL market orders. Orders are transmitted one at a time, with a short delay inserted to avoid hitting IB pacing limits. As a quant trader: in thin markets, slippage is a concern; using a limit order with VWAP execution is a reasonable alternative, but it’s not included in this first version.

Each close-out is logged to the terminal, including symbol, expiry, currency, exchange, position size, and multiplier. This provides transparency for each action taken and a final opportunity to manually monitor whether positions are being closed as expected.

In [ ]:
import os
import glob
import time
import threading
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order


class IBPositionCloser(EWrapper, EClient):
    def __init__(self, contract_lookup):
        EClient.__init__(self, self)
        self.open_positions = []
        self.nextOrderId = None
        self.positions_done = threading.Event()
        self.contract_lookup = contract_lookup

    def nextValidId(self, orderId):
        self.nextOrderId = orderId

    def position(self, account, contract, position, avgCost):
        if contract.secType == "FUT" and position != 0:
            self.open_positions.append({
                "symbol": contract.symbol,
                "exchange": contract.exchange,
                "currency": contract.currency,
                "expiry": contract.lastTradeDateOrContractMonth,
                "multiplier": int(float(contract.multiplier)) if contract.multiplier else 1,
                "position": position,
                "contract": contract
            })

    def positionEnd(self):
        print(f"\n✅ Finished receiving open positions. Total futures positions: {len(self.open_positions)}")
        self.positions_done.set()

    def create_market_order(self, action, quantity):
        order = Order()
        order.action = action
        order.totalQuantity = abs(int(quantity))
        order.orderType = "MKT"
        order.transmit = True
        return order

    def close_positions(self):
        for pos in self.open_positions:
            symbol = pos["symbol"]
            expiry = str(pos["expiry"])[:6]
            currency = pos["currency"]
            multiplier = float(pos["multiplier"])

            # If exchange is missing, try to resolve from TWS basket
            if not pos["exchange"] or pos["exchange"].strip() == "":
                key = (symbol, expiry, currency, multiplier)
                if key in self.contract_lookup:
                    pos["contract"].exchange = self.contract_lookup[key]
                    print(f"🔁 Filled missing exchange for {symbol}: {self.contract_lookup[key]}")
                else:
                    print(f"❌ Cannot place order for {symbol}: missing exchange and no match in basket")
                    continue
            else:
                pos["contract"].exchange = pos["exchange"]

            action = "SELL" if pos["position"] > 0 else "BUY"
            order = self.create_market_order(action, pos["position"])

            print(
                f"❗ Closing: {action} {abs(pos['position'])}x "
                f"{symbol} | Exp: {pos['expiry']} | Exch: {pos['contract'].exchange} | "
                f"Cur: {currency} | Mult: {multiplier}"
            )

            self.placeOrder(self.nextOrderId, pos["contract"], order)
            self.nextOrderId += 1
            time.sleep(0.5)


# === Load latest TWS basket ===
tws_folder = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\3.IB_Trades"
tws_path = sorted(glob.glob(os.path.join(tws_folder, "TWS_Basket_*.csv")))[-1]
tws_df = pd.read_csv(tws_path)

# === Normalize fields ===
tws_df['LastTradingDayOrContractMonth'] = tws_df['LastTradingDayOrContractMonth'].astype(str).str[:6]
tws_df['Multiplier'] = tws_df['Multiplier'].astype(float)

# === Build lookup: (symbol, expiry, currency, multiplier) → exchange ===
contract_lookup = {
    (row['Symbol'], str(row['LastTradingDayOrContractMonth']), row['Currency'], float(row['Multiplier'])): row['Exchange']
    for _, row in tws_df.iterrows()
}


# === Run the closer ===
closer = IBPositionCloser(contract_lookup)
closer.connect("127.0.0.1", 7497, clientId=88)
threading.Thread(target=closer.run, daemon=True).start()

# === Wait for IB connection ===
while closer.nextOrderId is None:
    time.sleep(1)

# === Request all open positions ===
closer.reqPositions()

# === Wait for all positions to be received ===
closer.positions_done.wait(timeout=10)

# === Close all matching positions ===
closer.close_positions()


In [ ]:
app.disconnect()

This review layer is mainly relevant for systems that use trailing stops or other reactive exit rules. It reduces the chance of submitting stale signals or misaligned exits that no longer make sense under the latest market conditions. While similar validation logic applies to _BUY_ orders, timing tends to matter more for exits—especially in leveraged or volatile futures markets—so the focus here is on the _SELL_ side.

With this, the execution logic for _SELL_ orders is complete. The next step is to refine the approach for _BUY_ orders.

#### 10.3.2. Execution Audit – Buy Orders
Before validating position sizing for BUY orders, we need to convert all contract risk exposures to a common currency. Since the execution capital is managed in USD, this step ensures that per-trade risk stays consistent across assets quoted in other currencies (e.g., EUR, JPY, AUD).

To retrieve current FX rates, a standalone script connects to Interactive Brokers and requests the most recent end-of-day midpoint prices for a fixed set of currency pairs. These include both direct and inverse crosses relative to USD (e.g., _EUR.USD, USD.JPY, AUD.USD_). The connection uses the IB API and requests a 1-day history for each pair using the MIDPOINT feed.

Each currency pair is stored in a lookup table with its most recent close. If a contract is quoted in USD, no conversion is applied. Otherwise, we attempt a direct match (e.g., _CHF.USD_) or fall back to the inverse (e.g., for _USD.JPY_, use 1 / _USDJPY_).

This FX dataframe (`fx_df`) is later referenced in the BUY order validation logic when calculating:

`RiskPerContract_AdjustedUSD = RiskPerContract * FX_Rate`

If no valid rate is found, the script defaults to assuming no conversion, though the position will be flagged accordingly.

In [ ]:
import time
import threading
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData


class IBFXFetcher(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = {}
        self.received = set()

    def historicalData(self, reqId: int, bar: BarData):
        self.data[reqId]['close'] = bar.close  # Save last close

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        self.received.add(reqId)


def create_fx_contract(pair_str):
    symbol, currency = pair_str.split(".")
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "CASH"
    contract.exchange = "IDEALPRO"
    contract.currency = currency
    return contract


# === FX Pairs to Retrieve ===
fx_pairs = [
    "EUR.USD", "AUD.USD", "USD.CAD", "CHF.USD",
    "GBP.USD", "USD.HKD", "USD.JPY", "KRW.USD", "USD.SGD"
]

# === Start IB Connection ===
app = IBFXFetcher()
app.connect("127.0.0.1", 7497, clientId=99)
threading.Thread(target=app.run, daemon=True).start()
time.sleep(1)

# === Request Last Close for Each FX Pair ===
for reqId, fx_pair in enumerate(fx_pairs):
    contract = create_fx_contract(fx_pair)
    app.data[reqId] = {"pair": fx_pair}
    app.reqHistoricalData(
        reqId=reqId,
        contract=contract,
        endDateTime="",
        durationStr="1 D",
        barSizeSetting="1 day",
        whatToShow="MIDPOINT",
        useRTH=0,
        formatDate=2,
        keepUpToDate=False,
        chartOptions=[]
    )

# === Wait for All Data ===
timeout = 30
start_time = time.time()
while len(app.received) < len(fx_pairs) and time.time() - start_time < timeout:
    time.sleep(0.5)

# === Build DataFrame ===
fx_rows = []
for reqId, info in app.data.items():
    fx_rows.append({
        "FX_Pair": info['pair'],
        "LastClose": info.get("close", None)
    })

fx_df = pd.DataFrame(fx_rows)
print(fx_df)

def get_fx_to_usd(currency, fx_df):
    if currency == "USD":
        return 1.0  # No conversion needed

    # Try direct conversion (e.g., EUR.USD)
    direct = fx_df[fx_df['FX_Pair'] == f"{currency}.USD"]
    if not direct.empty:
        return direct['LastClose'].values[0]

    # Try inverse (e.g., USD.JPY → 1 / USDJPY)
    inverse = fx_df[fx_df['FX_Pair'] == f"USD.{currency}"]
    if not inverse.empty and inverse['LastClose'].values[0] != 0:
        return 1 / inverse['LastClose'].values[0]

    # Not found
    return None

In [ ]:
app.disconnect()

This setup allows daily pricing of international contracts in a uniform risk currency without requiring real-time FX feeds or continuous data streams. It's simple, consistent with the execution timing (post-close), and sufficient for EoD strategies.

Now we need validation for BUY-side trades before they are executed. The goal is to check that each order still makes sense based on the latest price and volatility data, and that position sizing stays within our predefined risk limits. All checks are run using data pulled directly from Interactive Brokers.

The process begins by loading the latest model-generated TWS basket and filtering it to keep only BUY orders. These represent new entries into the portfolio. Each contract is then reconstructed using its key identifiers: symbol, expiry, exchange, currency, and multiplier.

A connection to IBKR is established using the API, and a 21-day historical price series is requested for each contract using the TRADES feed. Once all data is received, we calculate:

- Latest Close Price

- 20-day Mean and Standard Deviation

- Lower Bollinger Band (BB-Lower-20-3)

This gives us the dynamic stop level the models uses to control drawdown. The stop distance is defined as the difference between the current price and the lower Bollinger Band. That value, scaled by the contract’s multiplier, gives the risk per contract in local currency.

Before final position sizing, each trade’s native risk is converted to USD using the most recent FX rate (retrieved in the previous step), so all contracts are compared on a like-for-like basis. At the same time, the account’s available USD balance is retrieved through the get_usd_cash_ibapi helper, which connects to IB, collects the cash snapshot, and returns the result. This balance is then used in the validation step to ensure that buy orders stay within the account’s funding and risk limits.

In [ ]:
import time
import threading
from ibapi.client import EClient
from ibapi.wrapper import EWrapper

class CashBalanceApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.usd_cash = None
        self._acct_code = None
        self._done = False

    # 1) Get account codes first
    def managedAccounts(self, accountsList: str):
        # pick the first account; adjust selection logic if you have multiple
        self._acct_code = accountsList.split(",")[0].strip()
        # start account updates stream
        self.reqAccountUpdates(True, self._acct_code)

    # 2) Receive account value rows (this includes CashBalance by currency)
    def updateAccountValue(self, key, val, currency, accountName):
        if key == "CashBalance" and (currency or "").upper() == "USD":
            try:
                self.usd_cash = float(val)
            except:
                pass  # leave as None if it can't be parsed

    # 3) IB signals the end of the initial snapshot
    def accountDownloadEnd(self, accountName):
        # stop the stream so we can exit cleanly
        self.reqAccountUpdates(False, accountName)
        self._done = True


def get_usd_cash_ibapi(client_id: int = 77, host: str = "127.0.0.1", port: int = 7497, timeout: int = 10):
    app = CashBalanceApp()
    app.connect(host, port, clientId=client_id)

    t = threading.Thread(target=app.run, daemon=True)
    t.start()

    # Give IB time to connect and send managed accounts; then wait for snapshot end
    start = time.time()
    while not app._done and time.time() - start < timeout:
        time.sleep(10)

    app.disconnect()
    return app.usd_cash

# === Example usage ===
capital = get_usd_cash_ibapi()
print("USD CashBalance:", capital)

In [ ]:
app.disconnect()

We then determine how many contracts fit within the fixed risk budget (0.5% of total capital). This is done with:

`contracts_to_trade = floor(max_risk / risk_per_contract_usd)`

Each trade’s USD-adjusted risk, allocation size, and the total portfolio impact are calculated and stored.

In [ ]:
import time
import pandas as pd
import numpy as np
import threading
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData
from IPython.display import display, HTML

# === IB API Wrapper Class ===
class MyTradingStrategy(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = {}
        self.received = set()

    def historicalData(self, reqId: int, bar: BarData):
        if 'closes' not in self.data[reqId]:
            self.data[reqId]['closes'] = []
        self.data[reqId]['closes'].append(bar.close)

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        self.received.add(reqId)

# === Contract builder ===
def create_futures_contract(symbol, last_trade_date, exchange, currency, multiplier):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "FUT"
    contract.lastTradeDateOrContractMonth = str(last_trade_date)[:6]
    contract.exchange = exchange
    contract.currency = currency
    contract.multiplier = str(int(float(multiplier)))
    return contract

# === Load TWS basket (filter to BUY only) ===
import glob
tws_path = sorted(glob.glob(r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\2.Zipline_IB_Trades\TWS_Basket_*.csv"))[-1]
tws_df = pd.read_csv(tws_path)
tws_df['LastTradingDayOrContractMonth'] = tws_df['LastTradingDayOrContractMonth'].astype(str).str[:6]
tws_df['Multiplier'] = tws_df['Multiplier'].astype(str)

# Keep only BUY rows
buys_df = tws_df[tws_df['Action'].str.upper() == 'BUY'].reset_index(drop=True)

if buys_df.empty:
    display(HTML("<b>No BUY orders found in the latest TWS basket.</b>"))

# === Start IB connection ===
app = MyTradingStrategy()
app.connect("127.0.0.1", 7497, clientId=65)
threading.Thread(target=app.run, daemon=True).start()
time.sleep(1)

# === Request historical data (BUYs only) ===
for reqId, row in buys_df.iterrows():
    contract = create_futures_contract(
        row["Symbol"],
        row["LastTradingDayOrContractMonth"],
        row["Exchange"],
        row["Currency"],
        row["Multiplier"]
    )
    app.data[reqId] = {
        "symbol": row["Symbol"],
        "contract_month": row["LastTradingDayOrContractMonth"],
        "exchange": row["Exchange"],
        "currency": row["Currency"],
        "multiplier": float(row["Multiplier"]),
        "closes": []
    }
    app.reqHistoricalData(
        reqId=reqId,
        contract=contract,
        endDateTime="",
        durationStr="21 D",
        barSizeSetting="1 day",
        whatToShow="TRADES",
        useRTH=0,
        formatDate=2,
        keepUpToDate=False,
        chartOptions=[]
    )

# === Wait for data (BUY count) ===
timeout = 60
start_time = time.time()
while len(app.received) < len(buys_df) and time.time() - start_time < timeout:
    time.sleep(0.5)

# === Risk constants ===
capital = get_usd_cash_ibapi()
risk_factor = 0.005
max_risk = capital * risk_factor

# === Compile BUY risk table (with FX-adjusted RiskPerContract in USD) ===
part3_rows = []

for reqId, info in app.data.items():
    closes = info.get("closes", [])
    if len(closes) >= 20:
        close = closes[-1]
        mean20 = np.mean(closes[-20:])
        std20 = np.std(closes[-20:])
        bb_lower = mean20 - 3 * std20

        stop_distance = max(0, close - bb_lower)
        risk_per_contract = stop_distance * info["multiplier"]  # native currency

        # === Load row from BUY basket
        row = buys_df.iloc[reqId].to_dict()
        quotation = row.get("Quotation", "")
        symbol = row.get("Symbol", "")

        # === ✅ Adjust LmtPrice for symbols quoted in cents in Norgate (IB uses dollars)
        if symbol in ["ZR", "HG", "SI"]:
            row["LmtPrice"] /= 100

        # === ✅ General "cents per" quotation rule
        if isinstance(quotation, str) and quotation.lower().startswith("cents per"):
            risk_per_contract /= 100

        # === 💱 FX conversion to USD
        # NOTE: assumes you have get_fx_to_usd(currency, fx_df) and fx_df defined elsewhere
        fx_rate = get_fx_to_usd(row["Currency"], fx_df)
        risk_per_contract_usd = risk_per_contract * fx_rate if fx_rate is not None else risk_per_contract

        # === 💵 USD Risk Sizing
        contracts_to_trade = int(np.floor(max_risk / risk_per_contract_usd)) if risk_per_contract_usd > 0 else 0
        risk_per_trade = risk_per_contract_usd * contracts_to_trade

        # === Deviation & mismatch checks
        lmt = row["LmtPrice"]
        pct_diff = ((close - lmt) / lmt) if lmt not in (0, None) else np.nan
        mismatch = contracts_to_trade != row["Quantity"]

        # === Append row to results
        part3_rows.append({
            "Action": row["Action"],
            "Quantity": row["Quantity"],
            "Symbol": row["Symbol"],
            "LastTradingDayOrContractMonth": row["LastTradingDayOrContractMonth"],
            "Currency": row["Currency"],
            "OrderType": row["OrderType"],
            "Multiplier": float(row["Multiplier"]),
            "Quotation": quotation,
            "LmtPrice": lmt,
            "Close": close,
            "PctDiff": round(pct_diff * 100, 2) if pd.notnull(pct_diff) else None,
            "Alert_Deviation": abs(pct_diff) > 0.005 if pd.notnull(pct_diff) else False,
            "Alert_MissingPrice": False if pd.notnull(close) else True,
            "BB_Lower_20_3": round(bb_lower, 2),
            "StopDist": round(stop_distance, 2),
            "RiskPerContract": round(risk_per_contract, 2),                 # native currency
            "RiskPerContract_AdjustedUSD": round(risk_per_contract_usd, 2), # ✅ USD-adjusted
            "FX_Conversion": round(fx_rate, 6) if fx_rate is not None else "N/A",
            "ContractsToTrade": contracts_to_trade,
            "RiskPerTrade": round(risk_per_trade, 2),
            "PositionMismatch": mismatch,
            "RiskOK": risk_per_trade <= max_risk
        })

part3_df = pd.DataFrame(part3_rows)

# === Styling (no SELL overrides) ===
group1 = ['Action', 'Quantity', 'Symbol', 'LastTradingDayOrContractMonth', 'Currency', 'OrderType', 'Multiplier', 'Quotation']
group2 = ['LmtPrice', 'Close', 'PctDiff', 'BB_Lower_20_3', 'StopDist']
group3 = ['RiskPerContract', 'ContractsToTrade', 'RiskPerTrade', 'RiskPerContract_AdjustedUSD', 'FX_Conversion']
group4 = ['PositionMismatch']
group5 = ['RiskOK']

def highlight_columns(val, colname):
    if colname in group1:
        return 'background-color: #e6f7ff'  # light blue
    elif colname in group2:
        return 'background-color: #fff7e6'  # light yellow
    elif colname in group3:
        return 'background-color: #e6ffe6'  # light green
    elif colname in group4:
        return 'background-color: #ffe6cc'  # light orange
    elif colname in group5:
        return 'background-color: #f0fff0'  # light gray-green
    else:
        return ''

styled = part3_df.style.apply(lambda row: [highlight_columns(row[c], c) for c in part3_df.columns], axis=1)

# === Scrollable display ===
html = styled.to_html()
scrollable_html = f"""
<div style="overflow-x: auto; white-space: nowrap; font-family: monospace; font-size: 13px;">
    {html}
</div>
"""
display(HTML("<h3>📊 PART 3 (BUYs only): Risk Sizing and Bollinger Band Stop</h3>"))
display(HTML(scrollable_html))


In [ ]:
app.disconnect()

The script checks for several potential execution mismatches:

- Price Drift: If the most recent close deviates by more than ±0.5% from the model’s `LmtPrice`, the trade is flagged `(Alert_Deviation = True)`.

- Size Discrepancy: If the model’s suggested contract count differs from the TWS basket quantity, it’s also flagged `(PositionMismatch = True)`.

- Invalid Price: If no close price is returned, the entry is marked as incomplete.

These results are compiled into a final report containing key trade fields (contract details, pricing, stop logic, FX rate, and risk calculations). The table is styled and rendered for manual review before any orders are submitted.


Morover, the validated long-side trades and prepares them for execution using Interactive Brokers’ Basket Trader. The objective is to convert model-generated signals into a structured CSV file that TWS can parse directly.

This script is the final step in preparing the validated BUY-side trades for execution through Interactive Brokers' TWS Basket Trader. After risk checks are completed and the number of contracts per instrument is sized according to volatility and FX-adjusted stop distance, we need to ensure that the output aligns with IB’s expected structure.

Once keys are normalized, the strategy output (which includes calculated fields like ContractsToTrade, adjusted stop risk, and price deviation alerts) is joined with the original metadata extracted from the Zipline-to-IB basket. This metadata provides necessary fields like the correct exchange code for each symbol, which isn’t available directly from Zipline and is required by IB for routing.

In [ ]:
import os
from datetime import datetime
import numpy as np
import pandas as pd

# ---------- Normalize join keys in both DataFrames ----------
def norm_mult(x):
    # robust: 200.0 -> "200", "200" -> "200"
    try:
        return str(int(float(x)))
    except Exception:
        return str(x)

def normalize_keys(df):
    out = df.copy()
    out["Symbol"] = out["Symbol"].astype(str).str.upper()
    out["LastTradingDayOrContractMonth"] = out["LastTradingDayOrContractMonth"].astype(str).str[:6]
    out["Currency"] = out["Currency"].astype(str).str.upper()
    out["OrderType"] = out["OrderType"].astype(str).str.upper()
    out["Multiplier"] = out["Multiplier"].apply(norm_mult)
    return out

# buys_df comes from your BUY filter earlier
buy_meta_cols = ["Symbol", "LastTradingDayOrContractMonth", "Exchange", "Currency", "OrderType", "Multiplier"]
buys_meta = normalize_keys(buys_df[buy_meta_cols].drop_duplicates())

p3_cols_needed = ["Symbol","LastTradingDayOrContractMonth","Currency","OrderType","Multiplier",
                  "Action","LmtPrice","ContractsToTrade"]
# keep other computed columns too; but ensure keys are normalized
part3_norm = normalize_keys(part3_df)

# ---------- Merge (dtypes are aligned now) ----------
df = part3_norm.merge(
    buys_meta,
    on=["Symbol", "LastTradingDayOrContractMonth", "Currency", "OrderType", "Multiplier"],
    how="left"
)

# ---------- Build final IB basket ----------
ACCOUNT_ID     = "DU1234567"   # <-- set your account
BASKET_TAG     = "TF_BUY"
TIME_IN_FORCE  = "DAY"
SECTYPE        = "FUT"
DIVPRT         = ""            # or "0" if your workflow needs it

# Quantity from ContractsToTrade; drop zero/NaN
df["Quantity"] = pd.to_numeric(df["ContractsToTrade"], errors="coerce").fillna(0).astype(int)
df = df[df["Quantity"] > 0].copy()

# LmtPrice numeric + optional rounding for TWS
df["LmtPrice"] = pd.to_numeric(df["LmtPrice"], errors="coerce")
df = df.dropna(subset=["LmtPrice"])
df["LmtPrice"] = df["LmtPrice"].round(2)

# Fill fixed columns
df["SecType"]      = SECTYPE
df["DivPrt"]       = DIVPRT
df["TimeInForce"]  = TIME_IN_FORCE
df["BasketTag"]    = BASKET_TAG
df["Account"]      = ACCOUNT_ID
today_tag          = datetime.now().strftime("%Y%m%d")
df["OrderRef"]     = f"{BASKET_TAG}_{today_tag}"

# Reorder columns as requested
ib_cols = [
    "Action","Quantity","Symbol","SecType","LastTradingDayOrContractMonth",
    "DivPrt","Exchange","Currency","TimeInForce","OrderType",
    "LmtPrice","BasketTag","Account","OrderRef","Multiplier"
]
final_ib = df[ib_cols].sort_values(["Symbol","LastTradingDayOrContractMonth"]).reset_index(drop=True)

# ---------- Save to your specified folder ----------
output_dir = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\3.IB_Trades"
os.makedirs(output_dir, exist_ok=True)
out_name = f"TWS_Basket_BUY_{today_tag}.csv"
out_path = os.path.join(output_dir, out_name)

final_ib.to_csv(out_path, index=False)
print(f"✅ IB BUY basket saved: {out_path}")
display(pd.read_csv(out_path))


The final DataFrame is sorted by symbol and expiry, and written out to a .csv file in the directory used for all IB-bound trade instructions. This file format is directly ingestible by TWS via the Basket Trader interface. It allows a quick visual inspection before submitting orders, which is especially helpful when dealing with 50 to 100 futures contracts in a daily portfolio rebalance. The output is consistent, repeatable, and keeps execution aligned with the model logic.

Lastly, this script takes care of the final link between the system-generated trade signals and live execution through Interactive Brokers. The process is fully automated, but structured in a way that keeps control in the hands of the trader.

Once the app connects to TWS (or IB Gateway), it scans `C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\3.IB_Trades` for the most recently modified .csv file. This ensures we always pick up the latest validated trade basket without hardcoding filenames. After confirming the file, the script reads each row and loops through the orders one by one.

For every trade in the file, it builds the contract object and the associated order, then sends it to IB using `placeOrder`. A half-second pause is introduced between each submission to prevent pacing violations and allow enough time for IB to assign order IDs. This approach ensures each order is submitted cleanly, in sequence, and with all required parameters present.

In [ ]:
import os
import glob
import time
import pandas as pd
import threading
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order

class FuturesOrderSender(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextOrderId = None

    def nextValidId(self, orderId):
        self.nextOrderId = orderId

    def create_futures_contract(self, symbol, expiry, exchange, currency):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "FUT"
        contract.lastTradeDateOrContractMonth = str(expiry)
        contract.exchange = exchange
        contract.currency = currency
        contract.includeExpired = False
        return contract

    def create_limit_order(self, action, quantity, lmt_price):
        order = Order()
        order.action = action
        order.totalQuantity = int(quantity)
        order.orderType = "LMT"
        order.lmtPrice = float(lmt_price)
        order.transmit = True
        return order

    def get_latest_csv_file(self, folder_path):
        csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
        if not csv_files:
            raise FileNotFoundError("No CSV files found in folder.")
        latest_file = max(csv_files, key=os.path.getmtime)
        print(f"Latest basket file: {latest_file}")
        return latest_file

    def send_orders_from_file(self, filename):
        df = pd.read_csv(filename)

        for i, row in df.iterrows():
            contract = self.create_futures_contract(
                symbol=row["Symbol"],
                expiry=str(row["LastTradingDayOrContractMonth"]),
                exchange=row["Exchange"],
                currency=row["Currency"]
            )
            order = self.create_limit_order(
                action=row["Action"],
                quantity=row["Quantity"],
                lmt_price=row["LmtPrice"]
            )

            print(f"Placing {order.action} LIMIT order for {row['Symbol']} "
                  f"({row['LastTradingDayOrContractMonth']}) at {order.lmtPrice}")
            self.placeOrder(self.nextOrderId, contract, order)
            self.nextOrderId += 1
            time.sleep(0.5)  # pacing safety

# Initialize and run the IB app
app = FuturesOrderSender()
app.connect("127.0.0.1", 7497, clientId=67)
threading.Thread(target=app.run, daemon=True).start()

# Wait for IB connection
while app.nextOrderId is None:
    print("Waiting for IB connection...")
    time.sleep(1)

# Use the latest CSV file from your local folder
local_folder = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\3.IB_Trades"
latest_csv = app.get_latest_csv_file(local_folder)

# Send the orders
app.send_orders_from_file(latest_csv)


In [ ]:
app.disconnect()

It’s worth noting that this script assumes all upstream steps—position sizing, FX adjustments, price validations—have already been done. It doesn’t apply any logic to modify or check the orders. Its sole job is to take the final order list and push it to IB, exactly as written, with visibility into what's being sent through simple print logs.